# Logic/Explanation for methods in OrderBook Class

## ``__init__``

Instantiates an OrderBook object with 3 data structures:

**orders** is an empty default dictionary which holds all limit orders. it is structured as 
    
    {price: {order_number: quantity}}
    
For instance, if the dictionary is initially empty, and the first line of the data is is an add of quantity 20 at price 10.00 with order_id 504, the order would look like this after the first line is processed:
   
    {10.00: {504: 20}}
    

**order_prices** is an empty default dictionary which just allows for easy lookup for the price of a given order. This is especially useful for modify events because it makes it easy to look up what the previous price of the order was, as that information is not given in a line item - it only provides a new price. It doesn't include quantity information. Using the same example as before, after one line, the order_prices dictionary would look like:

    {504: 10.00}
    

**current_prices** is a list of all the different ask or bid prices at a given time. It starts with 0 because there have been no orders, however, after the first line, it would look like:

    [10.00]
    
This is useful because mid price is calculated as the midpoint between the lowest ask-price and the highest bid-price - so we sort this list to find either the lowest ask or highest bid, depending on which dictionary it is

## ``change_book(row)``

This method determines what kind of action should be done based on the 'book_event_type' column of a given row. Possible book_event_types are:
* 'A' for Add: add a new order to the OrderBook
* 'M' for Modify: change an existing order in the OrderBook
* 'C' for Cancel: cancel an existing order in the OrderBook
* 'T' for Trade: a trade has been executed on a previously existing order.

## ``add_order(row)``

* Add to the `orders` dictionary at the price -> an order with a certain quantity
* Add to the `order_prices` dictionary an order -> price



## ``modify_order(row)``

* Find the previous price of the order in the `order_prices` dictionary.
* Delete the previous order from the book, found by using the previous price.
* Create a new order at the new price with the new quantity
* Change the price of the order in the `order_prices` dictionary.
* Check if there are any more orders or shares requested at the previous price, and if not, delete the previous price, as it could effect what is the 'best ask/bid' price if there are no shares requested.

## ``cancel_order(row)``

* Find the order based on the price/order_id, and remove it from `orders`.
* Delete the order from `order_prices`.
* Check if there are any more orders or shares requested at the cancelled order's price, and if not, delete that price from `orders`.

## ``trade_order(row)``

* Find the previous price of the order in the `order_prices` dictionary (this needs to be done because sometimes a trade happens at a different price than was previously recorded).
* Change the quantity in `orders` to the value in the "aux_quantity" column. "aux_quantity" indicates how many shares are left over in an order after a trade. If "aux_quantity" is 0, then the order can be removed. Otherwise, it remains, just with fewer shares requested.
* If after a trade there are no more shares requested at a price across all orders, that price is removed.

## ``update_current_prices(side)``

* Sorts the prices in `orders`. 
* If it is a Bid order book, we want the highest prices first - that is the highest price someone is willing to pay, then the second highest price someone is willing to pay, etc.
* If it is an Ask order book, we want the lowest prices first - that is, the lowest prices someone is willing to sell for, etc.